## Finding Herschel `OBSID` for a sky source

Using the TAP interface to the Herschel Science Archive

In [ ]:
from astropy

In [ ]:
#
ra = "12h10m32.5s"
dec= "+39d24m20.0s"
c = SkyCoord(ra, dec, frame='icrs')

In [ ]:
#
#

radius = 15.0/60.0 # 15 arcmin circle
tap_cat = "v_public_observations"
circle = f"circle('ICRS',{c.ra.value:.4f},{c.dec.value:.4f},{radius})"
dist = f"DISTANCE(POINT('ICRS',t2.ra,t2.dec),POINT('ICRS',{c.ra.value:.4f},{c.dec.value:.4f}))"
# 
print (f"Search in {tap_cat} for observations intersecting with {circle}")
#
tap_url = "http://nxsa.esac.esa.int/tap-server/tap/sync?REQUEST=doQuery&LANG=ADQL&FORMAT=csv&"
# now combine two tables to include the observing mode
# instrument_mode_oid for EPN is from 60 to 69, will only search for those
#
query = f"QUERY=select top 100 t2.observation_id,t2.ra,t2.dec,t2.revolution,t2.duration,t3.mode_friendly_name" +\
    " from v_exposure as t1, v_public_observations as t2, v_instrument_mode as t3" + \
    " WHERE " + \
    "(t1.observation_oid = t2.observation_oid) and " +\
    "(t1.instrument_mode_oid BETWEEN 60 and 69) and " +\
    "(t1.instrument_mode_oid = t3.instrument_mode_oid) and " +\
    f"(1=intersects(observation_fov_scircle,{circle}))"
# this one works
#query = f"QUERY=select top 100 t2.observation_id,t2.ra,t2.dec,t2.revolution,t2.duration,t1.instrument_mode_oid" +\
#    " from v_exposure as t1, v_public_observations as t2" + \
#    " where t1.observation_oid = t2.observation_oid and (t1.instrument_mode_oid BETWEEN 60 and 69)" +\
#    f" and 1=intersects(observation_fov_scircle,{circle})"

#    " where t1.observation_oid = t2.observation_oid and (t1.instrument_mode_oid >= 60 and t1.instrument_mode_oid <= 69)" +\
#query = f"QUERY=SELECT+top+10+ra,dec,observation_id,revolution,duration+FROM+{tap_cat}" +\
#    f"+WHERE+1=intersects(observation_fov_scircle,{circle})"
xreq = tap_url + query
with requests.get(xreq) as r:
    jx = r.content.decode()